In [67]:
from numpy import *
from scipy import *
from scipy.sparse import *

In [3]:
tracks_with_tags=load("tracks_with_tags.npy")
tracks_with_tags


array([[2972914,   54087,    1757,    1718,  116712,  189631],
       [2750239,  189631,    3424,  177424,   46208,  205245],
       [1550729,   54087,  109806,   46869,  183258,   54337],
       ..., 
       [2584455,       0,       0,       0,       0,       0],
       [2299706,       0,       0,       0,       0,       0],
       [2739985,       0,       0,       0,       0,       0]], dtype=int32)

In [4]:
tag_indexed_byidtrack = zeros((amax(tracks_with_tags[:,0])+1, 5))

transfer = lambda x, y: where(tracks_with_tags[:,0]==x)
    
for x in tracks_with_tags[:,0]:
    i = where(tracks_with_tags[:,0]==x)
    tag_indexed_byidtrack[x] = tracks_with_tags[i,1:]




KeyboardInterrupt: 

In [5]:
tag_indexed_byidtrack.shape

(5018275, 5)

In [6]:
tag_indexed_byidtrack[1550729]

array([  54087.,  109806.,   46869.,  183258.,   54337.])

Ho creato un array che contiene tutti i tag delle canzoni con index l'id: 
tag_indexed_byidtrack[id] = [tags]

sveltirà di molto i calcoli successivi

In [7]:
playlists_with_tracks=load("playlists_with_tracks.npy")
playlists_with_tracks

array([[10024884,  2879006,  1532328, ...,        0,        0,        0],
       [10624787,  2090319,  2152295, ...,        0,        0,        0],
       [ 4891851,  2621486,  2895498, ...,        0,        0,        0],
       ..., 
       [11369546,   416692,  1587449, ...,        0,        0,        0],
       [ 7939535,  2788577,  3090083, ...,        0,        0,        0],
       [  297021,  2789282,  2154608, ...,        0,        0,        0]])

In [8]:
tracklist_by_playlist = delete(playlists_with_tracks, 0, 1)
tracklist_by_playlist

array([[2879006, 1532328, 3027673, ...,       0,       0,       0],
       [2090319, 2152295, 3230559, ...,       0,       0,       0],
       [2621486, 2895498, 1369381, ...,       0,       0,       0],
       ..., 
       [ 416692, 1587449, 1057389, ...,       0,       0,       0],
       [2788577, 3090083, 1609308, ...,       0,       0,       0],
       [2789282, 2154608,  973448, ...,       0,       0,       0]])

In [9]:
maxNumTracks=tracklist_by_playlist.shape[1]
maxNumTracks

192

In [10]:
highestTag=276614 # trovato ciclando sui tag
maxSize=highestTag+1

In [26]:
A=lil_matrix((10000,maxSize))
for x,tracks in enumerate(tracklist_by_playlist):
    tracks=trim_zeros(tracks)
    alltags=[]
    for t in tracks:
        tags = tag_indexed_byidtrack[t]
        alltags+=list(tags)
    tagsArray=(array(alltags)).astype(int)
    tagsCount=bincount(tagsArray)#posizione = tagID, contenuto = conteggio di quel tag
    tagsCount[0]=0#altrimenti venivano contati anche gli zeri
    tagsCount=resize(tagsCount,(maxSize,))
    A[x]=tagsCount

# A è una matrice sparsa, con 1 riga per playlist e sulla riga ci sono i count dei tag indicizzati per tag

In [78]:
A

<10000x276615 sparse matrix of type '<class 'numpy.float64'>'
	with 559049 stored elements in LInked List format>

In [69]:
A[0]

<1x276615 sparse matrix of type '<class 'numpy.float64'>'
	with 66 stored elements in LInked List format>

In [70]:
A[0].data

array([ list([1.0, 1.0, 2.0, 1.0, 1.0, 1.0, 22.0, 1.0, 20.0, 5.0, 1.0, 2.0, 1.0, 20.0, 5.0, 52.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 6.0, 1.0, 1.0, 1.0, 1.0, 53.0, 6.0, 1.0, 12.0, 1.0, 1.0, 1.0, 1.0, 2.0, 1.0, 1.0, 2.0, 1.0, 10.0, 1.0, 1.0, 1.0, 1.0, 1.0, 35.0, 2.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 2.0, 1.0, 1.0, 1.0, 2.0])], dtype=object)

In [77]:
A[1]

<1x276615 sparse matrix of type '<class 'numpy.float64'>'
	with 71 stored elements in LInked List format>

In [90]:
A_csc=csc_matrix(A)

In [91]:
A_csc

<10000x276615 sparse matrix of type '<class 'numpy.float64'>'
	with 559049 stored elements in Compressed Sparse Column format>

In [92]:
save_npz("playlists_with_tags.npz",A_csc)

In [93]:
B=load_npz("playlists_with_tags.npz")

In [94]:
B

<10000x276615 sparse matrix of type '<class 'numpy.float64'>'
	with 559049 stored elements in Compressed Sparse Column format>

In [96]:
B[0] # funziona!

<1x276615 sparse matrix of type '<class 'numpy.float64'>'
	with 66 stored elements in Compressed Sparse Column format>